End to End pyspark project

-> Google Playstore data

AGENDA:
we have google playstore dataset containing information of different apps installed rating and version and other details and we are going to do analysis based on the data we have

1. Find out top 10 reviews given to the apps
2. Top 10 installs apps and distribution of type [free/paid]
3. Category wise distribution of installed apps
4. Top free rating apps


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5f9c9b1a05272959df240ea033028351414a0427b8e348ceb302d7110c30fa51
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [21]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType,IntegerType
from pyspark.sql.functions import *

Create DatatFrame

In [22]:
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

In [23]:
df = spark.read.csv(path='/content/googleplaystore.csv', sep=",", header=True, escape='"', inferSchema=True)


In [24]:
df.count()

10841

In [25]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



Check Schema

In [26]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Data Cleaning


In [27]:
df = df.drop("size","Content Rating","Last Updated","Android Ver")

In [28]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|Current Ver|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|      1.0.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|      2.0.0|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
only showing top 2 rows



In [29]:
df= df.drop("Current Ver")

In [30]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [31]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [32]:
from pyspark.sql.functions import regexp_replace,col

df = df.withColumn("Reviews", col("reviews").cast(IntegerType()))\
.withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
              .withColumn("Price",regexp_replace(col("price"),"[$]",""))\
              .withColumn("Price",col("Price").cast(IntegerType()))

In [33]:
df.show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



In [62]:
df.createOrReplaceTempView("apps")

In [36]:
df1 = spark.sql("select * from apps")

In [38]:
df1.show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

Find out top 10 reviews given to the apps

In [41]:
df1 = spark.sql("SELECT App, SUM(Reviews) AS TotalReviews FROM apps GROUP BY App ORDER BY TotalReviews DESC")

In [43]:
df1.show()

+--------------------+------------+
|                 App|TotalReviews|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
|     Viber Messenger|    56675481|
|UC Browser - Fast...|    53140694|
|             YouTube|    51278853|
|        Temple Run 2|    48710930|
|Sniper 3D Gun Sho...|    46022233|
|      My Talking Tom|    44668928|
|Duolingo: Learn L...|    44047832|
|       Google Photos|    43423827|
|Clean Master- Spa...|    42916526|
|                 Pou|    41939801|
+--------------------+------------+
only showing top 20 rows



Top 10 installs apps and distribution of type [free/paid]

In [52]:
df1 = spark.sql("SELECT App, Type, SUM(CAST(Installs AS INT)) AS TotalInstalls FROM apps GROUP BY App, Type ORDER BY TotalInstalls DESC")
df1.show()


+--------------------+----+-------------+
|                 App|Type|TotalInstalls|
+--------------------+----+-------------+
|      Subway Surfers|Free|   6000000000|
|           Instagram|Free|   4000000000|
|        Google Drive|Free|   4000000000|
|            Hangouts|Free|   4000000000|
|       Google Photos|Free|   4000000000|
|         Google News|Free|   4000000000|
|    Candy Crush Saga|Free|   3500000000|
|  WhatsApp Messenger|Free|   3000000000|
|               Gmail|Free|   3000000000|
|        Temple Run 2|Free|   3000000000|
|Skype - free IM &...|Free|   3000000000|
|Google Chrome: Fa...|Free|   3000000000|
|Messenger – Text ...|Free|   3000000000|
|Maps - Navigate &...|Free|   3000000000|
|     Viber Messenger|Free|   2500000000|
|   Google Play Games|Free|   2000000000|
|            Facebook|Free|   2000000000|
|            Snapchat|Free|   2000000000|
|imo free video ca...|Free|   2000000000|
|  Google Street View|Free|   2000000000|
+--------------------+----+-------

Category wise distribution of installed apps

In [53]:
df1 = spark.sql("select Category, sum(Installs) from apps group by 1 order by 2 desc")
df1.show()



+-------------------+-------------+
|           Category|sum(Installs)|
+-------------------+-------------+
|               GAME|  35086024415|
|      COMMUNICATION|  32647276251|
|       PRODUCTIVITY|  14176091369|
|             SOCIAL|  14069867902|
|              TOOLS|  11452771915|
|             FAMILY|  10258263505|
|        PHOTOGRAPHY|  10088247655|
| NEWS_AND_MAGAZINES|   7496317760|
|   TRAVEL_AND_LOCAL|   6868887146|
|      VIDEO_PLAYERS|   6222002720|
|           SHOPPING|   3247848785|
|      ENTERTAINMENT|   2869160000|
|    PERSONALIZATION|   2325494782|
|BOOKS_AND_REFERENCE|   1921469576|
|             SPORTS|   1751174498|
| HEALTH_AND_FITNESS|   1583072512|
|           BUSINESS|   1001914865|
|            FINANCE|    876648734|
|          EDUCATION|    871452000|
|MAPS_AND_NAVIGATION|    724281890|
+-------------------+-------------+
only showing top 20 rows



Top free rating apps

In [68]:
df1 = spark.sql("SELECT App, SUM(CASE WHEN Type = 'paid' THEN Price ELSE 0 END) AS Total_Price FROM apps GROUP BY App ORDER BY Total_Price DESC")
df1.show()



+--------------------+-----------+
|                 App|Total_Price|
+--------------------+-----------+
|Google Chrome: Fa...|          0|
|free video calls ...|          0|
|Toddler Learning ...|          0|
|             MyChart|          0|
|Davis's Drug Guid...|          0|
|    Diabetes Testing|          0|
|Mercari: The Sell...|          0|
|Find&Save - Local...|          0|
|                SNCF|          0|
|Learn the letters...|          0|
|Nigeria News NAIJ...|          0|
|    Basketball Stars|          0|
|          C Examples|          0|
|            Q Wunder|          0|
|         Q-Ticketing|          0|
|Learn R Language ...|          0|
| Al-Quran Al-Muallim|          0|
|THE KING OF FIGHT...|          0|
|Ay Yıldız Analog ...|          0|
|A-Z Punjabi Songs...|          0|
+--------------------+-----------+
only showing top 20 rows

